# Baseline Regression Models (Windowed Data)

In [1]:
import numpy as np # Use version 1.x not 2.x
import random
import pandas as pd

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [2]:
import pickle

def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    records = []
    for company, periods in data.items():
        for period, (features, rating) in periods.items():
            record = {
                'company': company,
                'period': period,
                **{f'feature_{i}': feature.item() for i, feature in enumerate(features)},
                'rating': rating.item()
            }
            records.append(record)

    return pd.DataFrame(records)

In [3]:
from sklearn.preprocessing import StandardScaler

train_df = load_data('./data/train_dict_windowed.pkl')
test_df = load_data('./data/test_dict_windowed.pkl')

X_train = train_df.drop(columns=['company', 'period', 'rating'])
y_train = train_df['rating']
X_test = test_df.drop(columns=['company', 'period', 'rating'])
y_test = test_df['rating']

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
train_df.info()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,rating
count,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,...,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.000000,13419.00000
mean,3.528188,3.914564,3.657012,4.302699,3.896321,3.724823,5.059759,3.612374,3.924858,3.508447,...,-0.038140,-0.917355,-0.078312,-0.083236,0.055194,-0.029182,0.332568,-0.558261,1.797081,9.54706
std,10.885916,10.727755,10.247111,15.239469,12.464015,12.297962,16.753642,8.434904,10.456192,11.480522,...,4.323687,51.917468,0.594742,0.663512,0.680137,129.240364,1.876188,91.322609,7.384274,3.16827
min,-0.308304,-0.266087,-0.290627,-0.289814,-0.295400,-0.205454,-0.277362,-0.203502,-0.233085,-215.762192,...,-271.892731,-4882.298340,-1.141994,-1.142238,-0.532463,-12999.674805,-40.224262,-9938.629883,-0.649376,0.00000
25%,0.132040,0.207085,0.229832,0.000000,0.167359,0.020665,0.160252,0.283941,0.267857,0.000000,...,-0.418189,-0.239244,-0.477208,-0.504659,-0.359399,0.006327,-0.201479,-0.250107,-0.154979,7.00000
50%,0.839737,0.876969,0.906918,0.608566,0.831367,0.768885,0.898005,0.954726,0.968969,0.792770,...,0.020812,0.160447,-0.199570,-0.172257,-0.130685,0.410631,0.193717,0.177438,0.161311,10.00000
75%,2.441248,3.410828,3.107784,2.633272,2.655834,2.604900,3.249289,3.205135,3.283310,3.202153,...,0.471509,0.613430,0.204816,0.191851,0.218999,1.221484,0.748354,0.701961,0.956126,12.00000
max,145.475204,151.195236,142.204163,289.723022,190.370804,197.842560,233.938110,96.085632,130.575867,245.519623,...,204.916534,1640.114136,4.421368,6.690714,9.021321,4006.946289,30.412418,697.988953,569.232544,22.00000


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score

models = {
    'Decision Tree': DecisionTreeRegressor(random_state=RANDOM_SEED),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED),
    'Linear Regression': LinearRegression(),
    'k-NN (k=1)': KNeighborsRegressor(n_neighbors=1),
    'SVR': SVR(),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_SEED),
    'LightGBM': lgb.LGBMRegressor(random_state=RANDOM_SEED, verbose=-1)
}

print('Train-Test Split Regression Results')
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_rounded = np.round(y_pred).astype(int)
    y_pred_rounded = np.clip(y_pred_rounded, 0, 23)

    mse = mean_squared_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred_rounded)

    print(f'{model_name:20}: Accuracy={accuracy:.4f}  MSE={mse:.4f}')

Train-Test Split Regression Results
Decision Tree       : Accuracy=0.1588  MSE=7.9957
Random Forest       : Accuracy=0.2331  MSE=4.5266
Linear Regression   : Accuracy=0.1499  MSE=5.5325
k-NN (k=1)          : Accuracy=0.1307  MSE=8.7916
SVR                 : Accuracy=0.1790  MSE=4.6468
XGBoost             : Accuracy=0.2127  MSE=4.7628
LightGBM            : Accuracy=0.2403  MSE=4.2061


### Predict newer data (20%) from older data (80%)

In [6]:
def split_by_period(df):
    split_index = int(len(df) * 0.8)
    train_data = df.iloc[:split_index]
    test_data = df.iloc[split_index:]
    return train_data, test_data

merged_df = pd.concat([train_df, test_df], ignore_index=True)
merged_df = merged_df.sort_values(by=['company', 'period'])

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for company, group in merged_df.groupby('company'):
    train_data, test_data = split_by_period(group)
    train_df = pd.concat([train_df, train_data])
    test_df = pd.concat([test_df, test_data])

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [7]:
train_df.iloc[41:46]

,company,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,rating
41,AAL,2020Q2,6.397942,20.959440,15.179953,15.751231,12.555426,3.893929,22.912008,24.393595,...,-8.993368,-14.530337,-0.707366,-0.522562,0.225215,-14.396945,-2.791513,10.533374,0.934059,12.0
42,AAL,2020Q3,5.439050,21.035950,14.881256,11.709346,12.031379,4.010077,21.430578,24.599485,...,-4.068741,-8.808525,-0.722517,-0.550122,0.165980,-8.626437,-3.491277,6.862294,1.184092,12.0
43,AAL,2020Q4,4.715318,20.877996,14.523072,10.953375,11.431134,3.830160,22.500435,24.718748,...,-2.642344,-6.427341,-0.773885,-0.620862,0.061735,-7.137291,-3.521091,4.897902,1.222114,12.0
44,AAL,2021Q1,9.358152,20.610624,16.032759,30.888725,6.761358,3.559145,21.061703,29.293434,...,-3.286706,-3.899826,-0.515904,-0.258778,0.595875,-6.887997,-1.521122,2.209769,1.375260,12.0
45,AAP,2010Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.855467,0.205086,-0.413703,-1.014619,-0.429032,0.493377,0.415742,1.222719,-0.361857,10.0


In [8]:
test_df.head()

,company,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,rating
0,AAL,2021Q2,8.340997,20.478172,15.584459,26.190260,7.776797,3.513597,19.382196,30.482496,...,-0.926600,-0.427103,-0.509899,-0.238318,0.627533,-7.407228,-0.421442,-0.477901,1.509752,12.0
1,AAL,2021Q3,7.467043,20.675615,15.390813,22.217646,7.848329,3.470326,19.364405,30.831976,...,-0.650341,-0.245818,-0.536629,-0.289740,0.542962,-7.247670,-0.295384,-0.820659,1.671445,12.0
2,AAL,2021Q4,12.376627,20.530413,17.071869,44.420910,8.754915,3.570533,20.270578,34.958687,...,-0.729365,-1.547125,-0.606601,-0.384312,0.397845,-7.153681,-1.279906,1.699241,1.922507,12.0
3,AAL,2022Q1,15.576209,20.224787,18.037573,56.781490,10.118683,3.868877,25.237936,34.572418,...,-1.072615,-2.484440,-0.675459,-0.477723,0.277068,-6.182828,-1.881923,2.639538,1.734946,12.0
4,AAL,2022Q2,13.153007,19.988266,17.018206,46.270180,11.286515,4.010077,22.174259,33.579163,...,-0.267140,-0.063466,-0.674590,-0.484475,0.249731,-6.522480,-0.030636,-1.386750,1.871140,12.0


In [9]:
X_train = train_df.drop(columns=['company', 'period', 'rating'])
y_train = train_df['rating']
X_test = test_df.drop(columns=['company', 'period', 'rating'])
y_test = test_df['rating']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('Predict newer data (20%) from older data (80%) \nRegression Results')
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_rounded = np.round(y_pred).astype(int)
    y_pred_rounded = np.clip(y_pred_rounded, 0, 23)

    mse = mean_squared_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred_rounded)

    print(f'{model_name:20}: Accuracy={accuracy:.4f}  MSE={mse:.4f}')

Predict newer data (20%) from older data (80%) 
Regression Results
Decision Tree       : Accuracy=0.4101  MSE=5.5814
Random Forest       : Accuracy=0.3462  MSE=2.5745
Linear Regression   : Accuracy=0.1519  MSE=8.9382
k-NN (k=1)          : Accuracy=0.5535  MSE=4.6763
SVR                 : Accuracy=0.2297  MSE=4.4319
XGBoost             : Accuracy=0.2996  MSE=2.6747
LightGBM            : Accuracy=0.2725  MSE=2.6983


### Predict the most recent ratings from all past data

In [10]:
merged_df = pd.concat([train_df, test_df], ignore_index=True)
merged_df = merged_df.sort_values(by=['company', 'period'])

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for company, group in merged_df.groupby('company'):
    test_data = group.iloc[-1:]
    train_data = group.iloc[:-1]
    train_df = pd.concat([train_df, train_data])
    test_df = pd.concat([test_df, test_data])

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [11]:
print(len(test_df))
print(merged_df['company'].nunique())

346
346


In [12]:
X_train = train_df.drop(columns=['company', 'period', 'rating'])
y_train = train_df['rating']
X_test = test_df.drop(columns=['company', 'period', 'rating'])
y_test = test_df['rating']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('Predict the most recent ratings from all past data \nRegression Results')
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_rounded = np.round(y_pred).astype(int)
    y_pred_rounded = np.clip(y_pred_rounded, 0, 23)

    mse = mean_squared_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred_rounded)

    print(f'{model_name:20}: Accuracy={accuracy:.4f}  MSE={mse:.4f}')

Predict the most recent ratings from all past data 
Regression Results
Decision Tree       : Accuracy=0.5809  MSE=3.8497
Random Forest       : Accuracy=0.4769  MSE=1.3676
Linear Regression   : Accuracy=0.1647  MSE=8.2763
k-NN (k=1)          : Accuracy=0.7341  MSE=2.3642
SVR                 : Accuracy=0.2428  MSE=4.2640
XGBoost             : Accuracy=0.4046  MSE=1.3566
LightGBM            : Accuracy=0.3035  MSE=1.8720


### Appendix: Cross-Validation

#### split by companies (GroupKFold)

In [13]:
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline

merged_df = pd.concat([train_df, test_df], ignore_index=True)

X = merged_df.drop(columns=['company', 'period', 'rating'])
y = merged_df['rating']

groups = merged_df['company']

gkf = GroupKFold(n_splits=5)
print('GroupKFold (5-Fold Cross-Validation) Regression Results')

for model_name, model in models.items():
    fold_accuracies = []
    fold_mses = []
    
    for train_index, test_index in gkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        
        y_pred = pipeline.predict(X_test)
        y_pred_rounded = np.round(y_pred).astype(int)
        y_pred_rounded = np.clip(y_pred_rounded, 0, 23)
        
        mse = mean_squared_error(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred_rounded)
        
        fold_mses.append(mse)
        fold_accuracies.append(accuracy)
    
    print(f'{model_name:20}: Accuracy={np.mean(fold_accuracies):.4f}  MSE={np.mean(fold_mses):.4f} ')

GroupKFold (5-Fold Cross-Validation) Regression Results
Decision Tree       : Accuracy=0.1461  MSE=8.4071 
Random Forest       : Accuracy=0.1860  MSE=4.5632 
Linear Regression   : Accuracy=0.1524  MSE=8.1606 
k-NN (k=1)          : Accuracy=0.1394  MSE=8.7420 
SVR                 : Accuracy=0.1872  MSE=5.0561 
XGBoost             : Accuracy=0.1912  MSE=4.7144 
LightGBM            : Accuracy=0.2119  MSE=4.3002 


#### split by companies (StratifiedGroupKFold)

In [14]:
from sklearn.model_selection import StratifiedGroupKFold

merged_df = pd.concat([train_df, test_df], ignore_index=True)

X = merged_df.drop(columns=['company', 'period', 'rating'])
y = merged_df['rating']

groups = merged_df['company']

sgkf = StratifiedGroupKFold()
print('StratifiedGroupKFold (5-Fold Cross-Validation) Regression Results')

for model_name, model in models.items():
    fold_accuracies = []
    fold_mses = []
    
    for train_index, test_index in sgkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        
        y_pred = pipeline.predict(X_test)
        y_pred_rounded = np.round(y_pred).astype(int)
        y_pred_rounded = np.clip(y_pred_rounded, 0, 23)
        
        mse = mean_squared_error(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred_rounded)
        
        fold_mses.append(mse)
        fold_accuracies.append(accuracy)
    
    print(f'{model_name:20}: Accuracy={np.mean(fold_accuracies):.4f}  MSE={np.mean(fold_mses):.4f} ')

StratifiedGroupKFold (5-Fold Cross-Validation) Regression Results


c:\Users\Ryo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:994: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Decision Tree       : Accuracy=0.1639  MSE=8.4605 


c:\Users\Ryo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:994: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Random Forest       : Accuracy=0.2104  MSE=4.4061 


...

SVR                 : Accuracy=0.1919  MSE=5.0156 


c:\Users\Ryo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:994: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


XGBoost             : Accuracy=0.2019  MSE=4.6529 


c:\Users\Ryo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:994: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


LightGBM            : Accuracy=0.2092  MSE=4.1258 


#### shuffle/split by periods (data leakage?)

In [15]:
from sklearn.model_selection import KFold

merged_df = pd.concat([train_df, test_df], ignore_index=True)

X = merged_df.drop(columns=['company', 'period', 'rating'])
y = merged_df['rating']

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
print('5-Fold Cross-Validation (with shuffle) Regression Results')

for model_name, model in models.items():
    fold_accuracies = []
    fold_mses = []
    
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        
        y_pred = pipeline.predict(X_test)
        y_pred_rounded = np.round(y_pred).astype(int)
        y_pred_rounded = np.clip(y_pred_rounded, 0, 23)
        
        mse = mean_squared_error(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred_rounded)
        
        fold_mses.append(mse)
        fold_accuracies.append(accuracy)
    
    print(f'{model_name:20}: Accuracy={np.mean(fold_accuracies):.4f}  MSE={np.mean(fold_mses):.4f} ')

5-Fold Cross-Validation (with shuffle) Regression Results
Decision Tree       : Accuracy=0.6409  MSE=2.9404 
Random Forest       : Accuracy=0.5381  MSE=1.1216 
Linear Regression   : Accuracy=0.1494  MSE=7.2169 
k-NN (k=1)          : Accuracy=0.6701  MSE=2.9285 
SVR                 : Accuracy=0.2592  MSE=3.7891 
XGBoost             : Accuracy=0.4939  MSE=1.0418 
LightGBM            : Accuracy=0.4013  MSE=1.3993 
